In [8]:
# %% [markdown]
# # Economic Prosperity Prediction Platform
# ## Simplified Implementation with Robust Dependencies

# First, let's install only the essential packages that will work reliably
!pip install pandas numpy matplotlib seaborn scikit-learn plotly dash

# If you need XGBoost (optional for this simplified version):
# !pip install xgboost

# For this example, we'll use sklearn's built-in models instead of TensorFlow/XGBoost
# to ensure maximum compatibility

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# %% [markdown]
# ## 2. Data Preparation - Robust Synthetic Data
# %%
# Create more realistic synthetic data
np.random.seed(42)
years = list(range(2000, 2023))
countries = ['USA', 'China', 'Germany', 'India', 'Brazil']

data = {
    'Year': np.random.choice(years, 500),
    'Country': np.random.choice(countries, 500),
    'GDP_Growth': np.random.normal(3, 1, 500),
    'Urbanization': np.random.uniform(30, 90, 500),
    'Education': np.random.uniform(2, 6, 500),
    'Technology': np.random.uniform(0.5, 3, 500),
    'Inequality': np.random.uniform(0.25, 0.55, 500),
    'Environment': np.random.uniform(50, 100, 500)  # Environmental quality index
}

df = pd.DataFrame(data)

# Add realistic relationships
df['GDP_Growth'] = df['GDP_Growth'] + 0.15*df['Technology'] - 0.1*df['Inequality'] - 0.05*(100 - df['Environment'])
df['Inequality'] = df['Inequality'] + 0.02*df['Urbanization'] - 0.015*df['Education']

# One-hot encode countries
df = pd.get_dummies(df, columns=['Country'], prefix='Country')

# Show cleaned data
df.head()

# %% [markdown]
# ## 3. Simplified Modeling Approach
# %%
# Split data
X = df.drop(['GDP_Growth', 'Inequality'], axis=1)
y_gdp = df['GDP_Growth']
y_inequality = df['Inequality']

X_train, X_test, y_gdp_train, y_gdp_test = train_test_split(X, y_gdp, test_size=0.2, random_state=42)
_, _, y_inequality_train, y_inequality_test = train_test_split(X, y_inequality, test_size=0.2, random_state=42)

# GDP Model - Using Random Forest (more robust than linear regression)
gdp_model = RandomForestRegressor(n_estimators=100, random_state=42)
gdp_model.fit(X_train, y_gdp_train)
gdp_pred = gdp_model.predict(X_test)
print(f'GDP Model RMSE: {np.sqrt(mean_squared_error(y_gdp_test, gdp_pred)):.3f}')

# Inequality Model - Using Linear Regression for interpretability
inequality_model = LinearRegression()
inequality_model.fit(X_train, y_inequality_train)
inequality_pred = inequality_model.predict(X_test)
print(f'Inequality Model RMSE: {np.sqrt(mean_squared_error(y_inequality_test, inequality_pred)):.3f}')

# %% [markdown]
# ## 4. Policy Simulation Function
# %%
def simulate_policy(base_df, country, tech_investment=0, education_spending=0, env_protection=0):
    """Simulate policy impacts without complex dependencies"""
    country_col = f'Country_{country}'
    scenario_df = base_df[base_df[country_col] == 1].copy()
    
    # Apply changes
    scenario_df['Technology'] = scenario_df['Technology'] * (1 + tech_investment/100)
    scenario_df['Education'] = scenario_df['Education'] * (1 + education_spending/100)
    scenario_df['Environment'] = scenario_df['Environment'] * (1 + env_protection/100)

    
    # Predict outcomes
    X_scenario = scenario_df.drop(['GDP_Growth', 'Inequality'], axis=1)
    gdp_impact = gdp_model.predict(X_scenario).mean() - scenario_df['GDP_Growth'].mean()
    inequality_impact = inequality_model.predict(X_scenario).mean() - scenario_df['Inequality'].mean()
    
    return {
        'GDP Impact': f"{gdp_impact:.2f}%",
        'Inequality Impact': f"{inequality_impact:.4f}",
        'Environmental Impact': f"+{env_protection}% quality"
    }

# Test simulation
simulate_policy(df, 'China', tech_investment=10, education_spending=5, env_protection=15)

# %% [markdown]
# ## 5. Lightweight Dashboard
# %%
# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Economic Policy Simulator"),
    
    html.Div([
        html.Label("Select Country:"),
        dcc.Dropdown(
            id='country-selector',
            options=[{'label': c, 'value': c} for c in countries],
            value='USA'
        ),
    ], style={'width': '30%', 'padding': '20px'}),
    
    html.Div([
        html.Label("Technology Investment Change (%):"),
        dcc.Slider(id='tech-slider', min=-10, max=30, step=5, value=0),
    ], style={'padding': '20px'}),
    
    html.Div([
        html.Label("Education Spending Change (%):"),
        dcc.Slider(id='edu-slider', min=-5, max=20, step=5, value=0),
    ], style={'padding': '20px'}),
    
    html.Div([
        html.Label("Environmental Protection (%):"),
        dcc.Slider(id='env-slider', min=0, max=30, step=5, value=0),
    ], style={'padding': '20px'}),
    
    html.Div(id='policy-results', style={
        'margin-top': '30px',
        'padding': '20px',
        'border': '1px solid #ddd',
        'border-radius': '5px'
    })
])

@app.callback(
    Output('policy-results', 'children'),
    [Input('country-selector', 'value'),
     Input('tech-slider', 'value'),
     Input('edu-slider', 'value'),
     Input('env-slider', 'value')]
)
def update_policy(country, tech, edu, env):
    results = simulate_policy(df, country, tech, edu, env)
    return html.Div([
        html.H3(f"Policy Impact for {country}"),
        html.P(f"Projected GDP Change: {results['GDP Impact']}"),
        html.P(f"Projected Inequality Change: {results['Inequality Impact']}"),
        html.P(f"Environmental Improvement: {results['Environmental Impact']}"),
        html.P("Note: Positive inequality change means worsening inequality")
    ])

# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=8050)
    

GDP Model RMSE: 1.086
Inequality Model RMSE: 0.088


In [9]:
def update_policy(country, tech, edu, env):
    print(f"Types: country={type(country)}, tech={type(tech)}, edu={type(edu)}, env={type(env)}")
    results = simulate_policy(df, country, tech, edu, env)
    return results

In [10]:
def update_policy(country, tech, edu, env):
    tech = float(tech) if tech is not None else 0
    edu = float(edu) if edu is not None else 0
    env = float(env) if env is not None else 0
    results = simulate_policy(df, country, tech, edu, env)
    return results

In [11]:
#app.run_server(debug=True, port=8080)  # Use a different port
#app.run_server(host='0.0.0.0', port=8050)

In [12]:
import dash_bootstrap_components as dbc

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([...])  # Replace html.Div with responsive containers

In [ ]:
# %% [markdown]
# # Advanced Economic Prosperity Prediction Platform
# ## Incorporating Urbanization, Globalization, and Individualism

# %%
# Install required packages
!pip install pandas numpy matplotlib seaborn scikit-learn plotly dash dash-bootstrap-components

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import plotly.express as px
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, Input, Output

# %% [markdown]
# ## 2. Enhanced Data Preparation
# %%
# Create more comprehensive synthetic data
np.random.seed(42)
years = list(range(1990, 2023))
countries = ['USA', 'China', 'Germany', 'India', 'Brazil', 'Iran', 'Russia', 'England', 'UAE']

# Base economic indicators
data = {
    'Year': np.random.choice(years, 1000),
    'Country': np.random.choice(countries, 1000),
    'GDP_Growth': np.random.normal(3, 1.5, 1000),
    'Urbanization_Rate': np.random.uniform(30, 95, 1000),
    'Education_Spending': np.random.uniform(1, 8, 1000),
    'Tech_Investment': np.random.uniform(0.5, 5, 1000),
    'Trade_Openness': np.random.uniform(10, 150, 1000),  # Globalization proxy
    'Individualism_Index': np.random.uniform(10, 90, 1000),  # Hofstede's scale
    'Environmental_Quality': np.random.uniform(40, 100, 1000),
    'Political_Stability': np.random.uniform(-2, 2, 1000)  # World Bank index
}

df = pd.DataFrame(data)

# Add realistic relationships between factors
df['GDP_Growth'] = (
    df['GDP_Growth'] 
    + 0.12 * df['Urbanization_Rate']
    + 0.08 * df['Tech_Investment']
    + 0.05 * df['Trade_Openness']
    - 0.03 * (100 - df['Environmental_Quality'])
    + 0.02 * df['Individualism_Index']
    + 0.04 * df['Political_Stability']
)

# Country-specific adjustments
country_adjustments = {
    'China': {'Urbanization_Rate': +15, 'Trade_Openness': +30},
    'UAE': {'Tech_Investment': +2, 'Individualism_Index': -15},
    'Iran': {'Political_Stability': -1, 'Trade_Openness': -20},
    'Russia': {'Political_Stability': -0.5, 'Individualism_Index': -10},
    'England': {'Individualism_Index': +15, 'Education_Spending': +1}
}

for country, adjustments in country_adjustments.items():
    mask = df['Country'] == country
    for col, val in adjustments.items():
        df.loc[mask, col] += val

# Add derived metrics
df['Productivity'] = 0.4*df['Tech_Investment'] + 0.3*df['Education_Spending'] + 0.3*df['Urbanization_Rate']
df['Sustainability_Index'] = 0.7*df['Environmental_Quality'] - 0.3*(100 - df['Urbanization_Rate'])

# One-hot encode countries
df = pd.get_dummies(df, columns=['Country'], prefix='Country')

# Show sample data
df.sample(5)

# %% [markdown]
# ## 3. Advanced Modeling with Key Factors
# %%
# Prepare data
targets = ['GDP_Growth', 'Productivity', 'Sustainability_Index']
X = df.drop(targets, axis=1)
y = df[targets]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train models
models = {
    'GDP Growth': RandomForestRegressor(n_estimators=150, random_state=42),
    'Productivity': RandomForestRegressor(n_estimators=100, random_state=42),
    'Sustainability': RandomForestRegressor(n_estimators=100, random_state=42)
}

for i, (name, model) in enumerate(models.items()):
    model.fit(X_train, y_train.iloc[:, i])
    pred = model.predict(X_test)
    print(f"{name} Model:")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test.iloc[:, i], pred)):.3f}")
    print(f"R²: {r2_score(y_test.iloc[:, i], pred):.3f}\n")

# %% [markdown]
# ## 4. Comprehensive Policy Simulation
# %%
def simulate_policy(country, urbanization=0, globalization=0, individualism=0, 
                   tech=0, education=0, environment=0):
    """Enhanced simulation with all key factors"""
    country_col = f'Country_{country}'
    if country_col not in df.columns:
        country_col = [c for c in df.columns if c.startswith('Country_')][0]
    
    scenario = df[df[country_col] == 1].copy()
    
    # Apply changes
    scenario['Urbanization_Rate'] *= (1 + urbanization/100)
    scenario['Trade_Openness'] *= (1 + globalization/100)
    scenario['Individualism_Index'] *= (1 + individualism/100)
    scenario['Tech_Investment'] *= (1 + tech/100)
    scenario['Education_Spending'] *= (1 + education/100)
    scenario['Environmental_Quality'] *= (1 + environment/100)
    
    # Update derived metrics
    scenario['Productivity'] = 0.4*scenario['Tech_Investment'] + 0.3*scenario['Education_Spending'] + 0.3*scenario['Urbanization_Rate']
    scenario['Sustainability_Index'] = 0.7*scenario['Environmental_Quality'] - 0.3*(100 - scenario['Urbanization_Rate'])
    
    # Predict outcomes
    X_scenario = scenario.drop(targets, axis=1, errors='ignore')
    results = {
        'GDP Growth': f"{models['GDP Growth'].predict(X_scenario).mean() - scenario['GDP_Growth'].mean():+.2f}%",
        'Productivity': f"{models['Productivity'].predict(X_scenario).mean() - scenario['Productivity'].mean():+.2f}",
        'Sustainability': f"{models['Sustainability'].predict(X_scenario).mean() - scenario['Sustainability_Index'].mean():+.2f}",
        'Key Drivers': {
            'Urbanization': f"{urbanization}% change",
            'Globalization': f"{globalization}% change",
            'Individualism': f"{individualism}% change"
        }
    }
    return results

# Test simulation
simulate_policy('Iran', urbanization=5, globalization=-10, individualism=2, education=8)

# %% [markdown]
# ## 5. Professional Dashboard with All Factors
# %%
# Initialize Dash app with Bootstrap
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])

app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("Advanced Economic Prosperity Simulator"), className="text-center my-4")),
    
    dbc.Row([
        dbc.Col([
            html.Label("Select Country:"),
            dcc.Dropdown(
                id='country-selector',
                options=[{'label': c, 'value': c} for c in countries],
                value='China'
            ),
        ], md=4),
        
        dbc.Col([
            html.Label("Time Horizon:"),
            dcc.Slider(id='years-slider', min=1, max=10, step=1, value=5,
                      marks={i: f"{i} yrs" for i in range(1, 11)})
        ], md=8)
    ], className="mb-4"),
    
    dbc.Tabs([
        dbc.Tab(label="Core Factors", children=[
            dbc.Row([
                dbc.Col([
                    html.Label("Urbanization Change (%):"),
                    dcc.Slider(id='urban-slider', min=-10, max=20, step=1, value=0)
                ]),
                dbc.Col([
                    html.Label("Globalization Change (%):"),
                    dcc.Slider(id='global-slider', min=-20, max=30, step=1, value=0)
                ])
            ], className="mb-3"),
            
            dbc.Row([
                dbc.Col([
                    html.Label("Individualism Change (%):"),
                    dcc.Slider(id='individual-slider', min=-5, max=15, step=1, value=0)
                ]),
                dbc.Col([
                    html.Label("Tech Investment Change (%):"),
                    dcc.Slider(id='tech-slider', min=-5, max=25, step=1, value=0)
                ])
            ])
        ]),
        
        dbc.Tab(label="Policy Levers", children=[
            dbc.Row([
                dbc.Col([
                    html.Label("Education Spending Change (%):"),
                    dcc.Slider(id='edu-slider', min=-5, max=20, step=1, value=0)
                ]),
                dbc.Col([
                    html.Label("Environmental Protection (%):"),
                    dcc.Slider(id='env-slider', min=0, max=30, step=1, value=0)
                ])
            ])
        ])
    ], className="mb-4"),
    
    dbc.Card([
        dbc.CardHeader("Simulation Results", className="h5"),
        dbc.CardBody([
            dbc.Row([
                dbc.Col(html.Div(id='gdp-result'), className="border-end"),
                dbc.Col(html.Div(id='productivity-result'), className="border-end"),
                dbc.Col(html.Div(id='sustainability-result'))
            ], className="text-center py-3"),
            html.Div(id='factor-analysis')
        ])
    ]),
    
    dbc.Row([
        dbc.Col(dcc.Graph(id='factor-importance'), width=8),
        dbc.Col(dcc.Graph(id='country-comparison'), width=4)
    ], className="mt-4")
], fluid=True)

@app.callback(
    [Output('gdp-result', 'children'),
     Output('productivity-result', 'children'),
     Output('sustainability-result', 'children'),
     Output('factor-analysis', 'children'),
     Output('factor-importance', 'figure'),
     Output('country-comparison', 'figure')],
    [Input('country-selector', 'value'),
     Input('urban-slider', 'value'),
     Input('global-slider', 'value'),
     Input('individual-slider', 'value'),
     Input('tech-slider', 'value'),
     Input('edu-slider', 'value'),
     Input('env-slider', 'value'),
     Input('years-slider', 'value')]
)
def update_dashboard(country, urban, global_, individual, tech, edu, env, years):
    # Scale impacts by time horizon
    scale = 1 + (years / 10)
    urban_adj = urban * scale
    global_adj = global_ * scale
    individual_adj = individual * scale
    
    # Run simulation
    results = simulate_policy(country, urban_adj, global_adj, individual_adj, tech, edu, env)
    
    # Create result cards
    gdp_card = [
        html.H4("GDP Impact", className="card-title"),
        html.H2(results['GDP Growth'], className=("text-success" if '+' in results['GDP Growth'] else "text-danger"))
    ]
    
    productivity_card = [
        html.H4("Productivity", className="card-title"),
        html.H2(results['Productivity'], className=("text-success" if '+' in results['Productivity'] else "text-danger"))
    ]
    
    sustainability_card = [
        html.H4("Sustainability", className="card-title"),
        html.H2(results['Sustainability'], className=("text-success" if '+' in results['Sustainability'] else "text-danger"))
    ]
    
    # Factor analysis
    analysis = [
        html.H5("Key Factor Impacts"),
        html.Ul([html.Li(f"{k}: {v}") for k, v in results['Key Drivers'].items()]),
        html.P(f"Projecting impacts over {years} year period")
    ]
    
    # Feature importance plot
    importance_fig = px.bar(
        x=X_train.columns,
        y=models['GDP Growth'].feature_importances_,
        labels={'x': 'Factor', 'y': 'Importance'},
        title='GDP Growth Factor Importance'
    ).update_layout(showlegend=False)
    
    # Country comparison radar chart
    country_data = df[[c for c in df.columns if c.startswith('Country_') + ['Urbanization_Rate', 'Trade_Openness', 'Individualism_Index']]]
    country_means = country_data.groupby([c for c in country_data.columns if c.startswith('Country_')]).mean().reset_index()
    country_means = country_means.melt(id_vars=[c for c in country_data.columns if c.startswith('Country_')], 
                                     var_name='Factor', value_name='Value')
    
    radar_fig = px.line_polar(
        country_means[country_means['Factor'] != 'Country_England'],  # Exclude one for clarity
        r='Value', 
        theta='Factor',
        color='Country_China',  # Just for illustration - would need proper grouping
        line_close=True,
        title='Cross-Country Factor Comparison'
    )
    
    return gdp_card, productivity_card, sustainability_card, analysis, importance_fig, radar_fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=8050)

GDP Growth Model:
RMSE: 1.752
R²: 0.733

Productivity Model:
RMSE: 0.455
R²: 0.994

Sustainability Model:
RMSE: 0.864
R²: 0.996



---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[15], line 287, in update_dashboard(
    country='China',
    urban=0,
    global_=0,
    individual=0,
    tech=0,
    edu=0,
    env=0,
    years=5
)
    279 importance_fig = px.bar(
    280     x=X_train.columns,
    281     y=models['GDP Growth'].feature_importances_,
    282     labels={'x': 'Factor', 'y': 'Importance'},
    283     title='GDP Growth Factor Importance'
    284 ).update_layout(showlegend=False)
    286 # Country comparison radar chart
--> 287 country_data = df[[c for c in df.columns if c.startswith('Country_') + ['Urbanization_Rate', 'Trade_Openness', 'Individualism_Index']]]
        df =      Year  GDP_Growth  Urbanization_Rate  Education_Spending  Tech_Investment  \
0    2018   13.560137          61.921222            3.721084         4.930945   
1    2004   17.195470          90.262191            2.682094  

In [16]:
# Install required packages
!pip install pandas dash plotly dash-bootstrap-components scikit-learn

import pandas as pd
import numpy as np
import plotly.express as px
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, Input, Output
from sklearn.ensemble import RandomForestRegressor

# =====================
# 1. DATA PREPARATION
# =====================
countries = ['USA', 'China', 'UK', 'Russia', 'UAE', 'Iran', 'Germany', 'India', 'Brazil']
np.random.seed(42)

data = {
    'Year': np.random.choice(range(2010, 2023), 1000),
    'Country': np.random.choice(countries, 1000),
    # Traditional factors
    'Tech_Investment': np.random.uniform(0.5, 5, 1000),
    'Education_Spending': np.random.uniform(1, 8, 1000),
    'Environmental_Protection': np.random.uniform(30, 95, 1000),
    # Modern factors
    'Urbanization_Rate': np.random.uniform(30, 95, 1000),
    'Trade_Openness': np.random.uniform(10, 150, 1000),  # Globalization proxy
    'Individualism_Index': np.random.uniform(10, 90, 1000),  # Hofstede scale
    # Outcomes
    'GDP_Growth': np.random.normal(3, 1.5, 1000)
}

df = pd.DataFrame(data)

# Add realistic relationships
df['GDP_Growth'] = (
    df['GDP_Growth'] 
    + 0.12 * df['Tech_Investment']
    + 0.08 * df['Education_Spending']
    + 0.05 * df['Environmental_Protection']
    + 0.07 * df['Urbanization_Rate']
    + 0.04 * df['Trade_Openness']
    + 0.03 * df['Individualism_Index']
)

# Country-specific adjustments
adjustments = {
    'USA': {'Individualism_Index': +15, 'Tech_Investment': +2},
    'UAE': {'Environmental_Protection': -10, 'Trade_Openness': +40},
    'Iran': {'Trade_Openness': -30, 'Tech_Investment': -1},
    'Russia': {'Individualism_Index': -10, 'Education_Spending': -0.5}
}

for country, adj in adjustments.items():
    mask = df['Country'] == country
    for col, val in adj.items():
        df.loc[mask, col] += val

# =====================
# 2. MODEL TRAINING
# =====================
X = pd.get_dummies(df.drop(['GDP_Growth', 'Country'], axis=1))
y = df['GDP_Growth']

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# =====================
# 3. DASH APP
# =====================
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN])

app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("Global Economic Prosperity Simulator"), className="text-center my-4")),
    
    dbc.Row(
        dbc.Col(
            html.Div(
                "Done By Ronak Baniabbasi | May 5, 2025",
                style={
                    'textAlign': 'center',
                    'color': 'gray',
                    'marginTop': '30px',
                    'fontSize': '0.9rem'
                }
            ),
            width=12
        )
    ),
    
    dbc.Row([
        dbc.Col([
            html.Label("Select Country:"),
            dcc.Dropdown(id='country', options=[{'label':c, 'value':c} for c in countries], value='China')
        ], md=4),
        dbc.Col([
            html.Label("Projection Years:"),
            dcc.Slider(id='years', min=1, max=10, value=5, marks={i: str(i) for i in range(1, 11)})
        ], md=8)
    ]),
    
    
    dbc.Tabs([
        dbc.Tab(label="Traditional Factors", children=[
            dbc.Row([
                dbc.Col([
                    html.Label("Technology Investment Change (%):"),
                    dcc.Slider(id='tech', min=-10, max=50, value=0)
                ]),
                dbc.Col([
                    html.Label("Education Spending Change (%):"),
                    dcc.Slider(id='edu', min=-10, max=30, value=0)
                ])
            ]),
            dbc.Row(dbc.Col([
                html.Label("Environmental Protection Change (%):"),
                dcc.Slider(id='env', min=-20, max=40, value=0)
            ]))
        ]),
        
        dbc.Tab(label="Modern Factors", children=[
            dbc.Row([
                dbc.Col([
                    html.Label("Urbanization Rate Change (%):"),
                    dcc.Slider(id='urban', min=-5, max=25, value=0)
                ]),
                dbc.Col([
                    html.Label("Globalization (Trade Openness) Change (%):"),
                    dcc.Slider(id='global', min=-30, max=40, value=0)
                ])
            ]),
            dbc.Row(dbc.Col([
                html.Label("Individualism Index Change (%):"),
                dcc.Slider(id='individual', min=-10, max=20, value=0)
            ]))
        ])
    ], className="my-4"),
    
    dbc.Card([
        dbc.CardHeader("Simulation Results", className="h4"),
        dbc.CardBody([
            dbc.Row([
                dbc.Col(html.Div(id='gdp-impact', className="h3 text-center")),
                dbc.Col(html.Div(id='factor-analysis'))
            ]),
            dcc.Graph(id='importance-plot')
        ])
    ]),
    
    dbc.Row([
        dbc.Col(dcc.Graph(id='country-comparison'), width=8),
        dbc.Col(dcc.Graph(id='factor-tradeoff'), width=4)
    ], className="mt-4")
])


@app.callback(
    [Output('gdp-impact', 'children'),
     Output('factor-analysis', 'children'),
     Output('importance-plot', 'figure'),
     Output('country-comparison', 'figure'),
     Output('factor-tradeoff', 'figure')],
    [Input('country', 'value'),
     Input('tech', 'value'),
     Input('edu', 'value'),
     Input('env', 'value'),
     Input('urban', 'value'),
     Input('global', 'value'),
     Input('individual', 'value'),
     Input('years', 'value')]
)
def update_dashboard(country, tech, edu, env, urban, global_, individual, years):
    # Scale policy impacts by time horizon
    scale = 1 + (years / 10)
    inputs = {
        'Tech_Investment': tech * scale,
        'Education_Spending': edu * scale,
        'Environmental_Protection': env * scale,
        'Urbanization_Rate': urban * scale,
        'Trade_Openness': global_ * scale,
        'Individualism_Index': individual * scale,
        'Year': 2023 + years
    }
    
    # Predict GDP impact
    input_df = pd.DataFrame([inputs])
    input_df = pd.get_dummies(input_df).reindex(columns=X.columns, fill_value=0)
    gdp_change = model.predict(input_df)[0] - df[df['Country'] == country]['GDP_Growth'].mean()
    
    # Create outputs
    gdp_result = html.Div([
        html.H4(f"Projected GDP Change in {years} Years:"),
        html.H2(f"{gdp_change:.2f}%", style={'color': 'green' if gdp_change > 0 else 'red'})
    ])
    
    factor_analysis = html.Ul([
        html.Li(f"Technology: {tech}%"),
        html.Li(f"Education: {edu}%"),
        html.Li(f"Environment: {env}%"),
        html.Li(f"Urbanization: {urban}%"),
        html.Li(f"Globalization: {global_}%"),
        html.Li(f"Individualism: {individual}%")
    ])
    
    # Visualizations
    importance = pd.DataFrame({'Factor': X.columns, 'Importance': model.feature_importances_})
    importance_fig = px.bar(importance, x='Factor', y='Importance', title='Factor Importance for GDP Growth')
    
    country_comparison = px.line_polar(
        df.groupby('Country').mean().reset_index(),
        r='GDP_Growth',
        theta='Country',
        title='GDP Growth Comparison'
    )
    
    tradeoff_fig = px.scatter(
        df,
        x='Individualism_Index',
        y='Environmental_Protection',
        color='GDP_Growth',
        hover_name='Country',
        title='Individualism vs Environmental Protection Trade-offs'
    )
    
    return gdp_result, factor_analysis, importance_fig, country_comparison, tradeoff_fig


    # Your name and date added here
    


if __name__ == '__main__':
    app.run(debug=True, port=8000)